In [1]:
%load_ext google.cloud.bigquery

In [2]:
import numpy as np 
import pandas as pd
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/shuang/Downloads/bqair-2b59936a9500.json"

# Big query helpers
from google.cloud import bigquery
from bq_helper import BigQueryHelper

# Import plotting libaries
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.io as pio
import geopandas
import shapely

# the Plotly in offline mode we can see map in the notebook.
from plotly.offline import download_plotlyjs,init_notebook_mode, plot,iplot
init_notebook_mode(connected=True)

In [ ]:
#getting CO daily summary data first:
QUERY = """
    SELECT
        state_code,
        county_code,
        date_local,
        arithmetic_mean,
        aqi
    FROM
      `bigquery-public-data.epa_historical_air_quality.co_daily_summary`
    WHERE
      poc = 1
      AND EXTRACT(YEAR FROM date_local) = 2015
    
        """
bq_assistant = BigQueryHelper("bigquery-public-data", "epa_historical_air_quality")
df_co = bq_assistant.query_to_pandas(QUERY) # getting data through big query to pandas
df_co.head()
df_co['location_code']=df_co['state_code']+df_co['county_code'] # combine state code and county code to be location code. the location code will be fips code, make it easy to make the next choropleth map.
df_co.head()
df_co.info() #we check how many rows and how many null values in this data.
df_co.isnull().sum(axis = 0) # we see aqi column has 96802 null values.
#we deal the null values and convert them into average value of that column:
for column in df_co.columns: 
    if df_co[column].dtype in ['float64', 'int64']: 
        df_co[column].fillna(df_co[column].mean(), inplace = True)
        
df_co.sample(10,random_state = 20) #check if nan was replaced by mean 
df_co_part=df_co.sample(30000, random_state = 50)
df_co_part.head()
#now we are visualizing the co aqi map:
colorscale = ["#f7fbff", "#ebf3fb", "#deebf7", "#d2e3f3", "#c6dbef", "#b3d2e9", "#9ecae1",
    "#85bcdb", "#6baed6", "#57a0ce", "#4292c6", "#3082be", "#2171b5", "#1361a9",
    "#08519c", "#0b4083", "#08306b"
]

fips = df_co_part['location_code'].tolist()
values = df_co_part['aqi'].tolist()
endpts = list(np.linspace(min(values), max(values), len(colorscale) - 1))

fig = ff.create_choropleth(
    fips = fips, values = values, scope = ['usa'],
    binning_endpoints = endpts, colorscale = colorscale,
    show_state_data = False,
    show_hover = True, centroid_marker = {'opacity': 0},
    asp = 2.9,
    title = 'average air quality index of co by county in usa of 2015',
    legend_title = 'aqi by county'
)

iplot(fig)

In [ ]:
#getting O3 daily data :
QUERY = """
    SELECT
        state_code,
        county_code,
        date_local,
        arithmetic_mean,
        aqi
    FROM
      `bigquery-public-data.epa_historical_air_quality.o3_daily_summary`
    WHERE
      poc = 1
      AND EXTRACT(YEAR FROM date_local) = 2015
    
        """
df_o3 = bq_assistant.query_to_pandas(QUERY)
df_o3.head()
df_o3.info()
df_o3.isnull().sum(axis = 0) # so there is no need to deal with non values in this data .
df_o3['location_code']=df_o3['state_code']+df_o3['county_code']
df_o3_part = df_o3.sample(20000, random_state = 50)
#now we are visualizing the o3 aqi map:
colorscale = ["#f7fbff", "#ebf3fb", "#deebf7", "#d2e3f3", "#c6dbef", "#b3d2e9", "#9ecae1",
    "#85bcdb", "#6baed6", "#57a0ce", "#4292c6", "#3082be", "#2171b5", "#1361a9",
    "#08519c", "#0b4083", "#08306b"
]

fips = df_o3_part['location_code'].tolist()
values = df_o3_part['aqi'].tolist()
endpts = list(np.linspace(min(values), max(values), len(colorscale) - 1))

fig = ff.create_choropleth(
    fips = fips, values = values, scope = ['usa'],
    binning_endpoints = endpts, colorscale = colorscale,
    show_state_data = False,
    show_hover = True, centroid_marker = {'opacity': 0},
    asp = 2.9,
    title = 'average air quality index of o3 by county in usa of 2015',
    legend_title = 'aqi by county'
)

iplot(fig)

In [ ]:
#getting pm25 data:
QUERY = """
    SELECT
        state_code,
        county_code,
        date_local,
        arithmetic_mean,
        aqi
    FROM
      `bigquery-public-data.epa_historical_air_quality.pm25_frm_daily_summary`
    WHERE
      poc = 1
      AND EXTRACT(YEAR FROM date_local) = 2015
    
        """
df_pm25 = bq_assistant.query_to_pandas(QUERY)
df_pm25['location_code']=df_pm25['state_code']+df_pm25['county_code']
df_pm25.head()
df_pm25.info() #check if the data has null values:
df_pm25.isnull().sum(axis = 0) # non null values
df_pm25_part = df_pm25.sample(30000, random_state = 50)
#now we are visualizing the pm25 aqi map:
colorscale = ["#f7fbff", "#ebf3fb", "#deebf7", "#d2e3f3", "#c6dbef", "#b3d2e9", "#9ecae1",
    "#85bcdb", "#6baed6", "#57a0ce", "#4292c6", "#3082be", "#2171b5", "#1361a9",
    "#08519c", "#0b4083", "#08306b"
]

fips = df_pm25_part['location_code'].tolist()
values = df_pm25_part['aqi'].tolist()
endpts = list(np.linspace(min(values), max(values), len(colorscale) - 1))

fig = ff.create_choropleth(
    fips = fips, values = values, scope = ['usa'],
    binning_endpoints = endpts, colorscale = colorscale,
    show_state_data = False,
    show_hover = True, centroid_marker = {'opacity': 0},
    asp = 2.9,
    title = 'average air quality index of pm25 by county in usa of 2015',
    legend_title = 'aqi by county'
)

iplot(fig)

In [ ]:
#getting pm10 data:
QUERY = """
    SELECT
        state_code,
        county_code,
        date_local,
        arithmetic_mean,
        aqi
    FROM
      `bigquery-public-data.epa_historical_air_quality.pm10_daily_summary`
    WHERE
      poc = 1
      AND EXTRACT(YEAR FROM date_local) = 2015
    
        """
bq_assistant = BigQueryHelper("bigquery-public-data", "epa_historical_air_quality")
df_pm10 = bq_assistant.query_to_pandas(QUERY)
df_pm10['location_code']=df_pm10['state_code']+df_pm10['county_code']
df_pm10.head()
df_pm10.isnull().sum(axis = 0) # non null values
df_pm10_part = df_pm10.sample(30000, random_state = 50)
#now we are visualizing the pm10 aqi map:
colorscale = ["#f7fbff", "#ebf3fb", "#deebf7", "#d2e3f3", "#c6dbef", "#b3d2e9", "#9ecae1",
    "#85bcdb", "#6baed6", "#57a0ce", "#4292c6", "#3082be", "#2171b5", "#1361a9",
    "#08519c", "#0b4083", "#08306b"
]

fips = df_pm10_part['location_code'].tolist()
values = df_pm10_part['aqi'].tolist()
endpts = list(np.linspace(min(values), max(values), len(colorscale) - 1))

fig = ff.create_choropleth(
    fips = fips, values = values, scope = ['usa'],
    binning_endpoints = endpts, colorscale = colorscale,
    show_state_data = False,
    show_hover = True, centroid_marker = {'opacity': 0},
    asp = 2.9,
    title = 'average air quality index of pm10 by county in usa of 2015',
    legend_title = 'aqi by county'
)

iplot(fig)

In [ ]:
#getting no2 data:
QUERY = """
    SELECT
        state_code,
        county_code,
        date_local,
        arithmetic_mean,
        aqi
    FROM
      `bigquery-public-data.epa_historical_air_quality.no2_daily_summary`
    WHERE
      poc = 1
      AND EXTRACT(YEAR FROM date_local) = 2015
    
        """
bq_assistant = BigQueryHelper("bigquery-public-data", "epa_historical_air_quality")
df_no2 = bq_assistant.query_to_pandas(QUERY)
df_no2['location_code']=df_no2['state_code']+df_no2['county_code']
df_no2.head()
df_no2.isnull().sum(axis = 0) # non null values
df_no2_part = df_no2.sample(30000, random_state = 50)
#now we are visualizing the no2 aqi map:
colorscale = ["#f7fbff", "#ebf3fb", "#deebf7", "#d2e3f3", "#c6dbef", "#b3d2e9", "#9ecae1",
    "#85bcdb", "#6baed6", "#57a0ce", "#4292c6", "#3082be", "#2171b5", "#1361a9",
    "#08519c", "#0b4083", "#08306b"
]

fips = df_no2_part['location_code'].tolist()
values = df_no2_part['aqi'].tolist()
endpts = list(np.linspace(min(values), max(values), len(colorscale) - 1))

fig = ff.create_choropleth(
    fips = fips, values = values, scope = ['usa'],
    binning_endpoints = endpts, colorscale = colorscale,
    show_state_data = False,
    show_hover = True, centroid_marker = {'opacity': 0},
    asp = 2.9,
    title = 'average air quality index of no2 by county in usa of 2015',
    legend_title = 'aqi by county'
)

iplot(fig)

In [ ]:
#getting so2 data:
QUERY = """
    SELECT
        state_code,
        county_code,
        date_local,
        arithmetic_mean,
        aqi
    FROM
      `bigquery-public-data.epa_historical_air_quality.so2_daily_summary`
    WHERE
      poc = 1
      AND EXTRACT(YEAR FROM date_local) = 2015
    
        """
bq_assistant = BigQueryHelper("bigquery-public-data", "epa_historical_air_quality")
df_so2 = bq_assistant.query_to_pandas(QUERY)
df_so2['location_code']=df_so2['state_code']+df_so2['county_code']
df_so2.head()
df_so2.isnull().sum(axis = 0) # aqi has lots of null values we will treat them below.
for column in df_so2.columns: 
    if df_so2[column].dtype in ['float64', 'int64']: 
        df_so2[column].fillna(df_so2[column].mean(), inplace = True)
df_so2_part = df_so2.sample(30000, random_state = 50)
#now we are visualizing the so2 aqi map:
colorscale = ["#f7fbff", "#ebf3fb", "#deebf7", "#d2e3f3", "#c6dbef", "#b3d2e9", "#9ecae1",
    "#85bcdb", "#6baed6", "#57a0ce", "#4292c6", "#3082be", "#2171b5", "#1361a9",
    "#08519c", "#0b4083", "#08306b"
]

fips = df_so2_part['location_code'].tolist()
values = df_so2_part['aqi'].tolist()
endpts = list(np.linspace(min(values), max(values), len(colorscale) - 1))

fig = ff.create_choropleth(
    fips = fips, values = values, scope = ['usa'],
    binning_endpoints = endpts, colorscale = colorscale,
    show_state_data = False,
    show_hover = True, centroid_marker = {'opacity': 0},
    asp = 2.9,
    title = 'average air quality index of so2 by county in usa of 2015',
    legend_title = 'aqi by county'
)

iplot(fig)